# Quality assurance and control of raster data

This notebook provides QA/QC analyses on the Minnesota digital elevation model and land cover rasters used for the final project.

This includes:
- Data cleaning
- Data range checks (spatial and non-spatial)
- Recommendations for data correction

All raster data is reprojected (if needed) to the WGS 84 coordinate system, as this is the coordinate system used within ArcGIS Online, where the final brown marmorated stinkbug locations map will be located.

In [14]:
# Import all required libraries
import requests
import zipfile
import io
import pandas as pd
import arcpy
import numpy
import json
import os

directory = 'C:\\Users\\15612\Documents\\Arc II\\Lab 2'

In [15]:
# Establish variables for project and map
project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]
spatial_ref = arcpy.SpatialReference(4326)

In [16]:
# Define a function to pull data from the Minnesota Geospatial Commons
def mn_geo_pull_and_unzip(url, directory, GDB_or_SHP):

    # Get GeoJSON from MN Geospatial Commons
    api = requests.get(url)
    json = api.json()
    
    # Use second list in the 'resources' key if data is in file geodatabase format
    if GDB_or_SHP == 'GDB':
        zip_link = requests.get(json['result']['resources'][1]['url'])
    
    # Use first list in the 'resources' key if data is in shapefile format
    if GDB_or_SHP == 'SHP':
        zip_link = requests.get(json['result']['resources'][2]['url'])
    
    # Get zipfile and extract
    z_file = zipfile.ZipFile(io.BytesIO(zip_link.content))
    z_file.extractall(directory)

In [17]:
# Run the created function using MN Geospatial Commons' CKAN API
mn_geo_pull_and_unzip('https://gisdata.mn.gov/api/3/action/package_show?id=bdry-state',directory + "\\Minn_State_Boundary","SHP")
mn_geo_pull_and_unzip('https://gisdata.mn.gov/api/3/action/package_show?id=bdry-mn-city-township-unorg',directory + '\\MN_City_Boundaries',"SHP")

# Add the data from the extracted files
m.addDataFromPath(directory + '\\Minn_State_Boundary\\Boundaries_of_Minnesota.shp')
m.addDataFromPath(directory + '\\MN_City_Boundaries\\city_township_unorg.shp')

In [18]:
# Define a quick function which reprojects vector data
def reproject(in_data,out_data):

    # Use 'Project' tool to reproject vector data
    arcpy.management.Project(
        in_dataset = in_data,
        out_dataset = out_data,
        out_coor_system = spatial_ref
    )
    
    # Delete old feature with incorrect projection
    arcpy.management.Delete(
            in_data = in_data
        )

In [19]:
# Run the tool on acquired data from the MN Geospatial Commons
reproject('Boundaries_of_Minnesota','Minnesota')
reproject('city_township_unorg','Cities')

In [22]:
# Pull and unzip DEM from MN Geospatial Commons
mn_geo_pull_and_unzip('https://gisdata.mn.gov/api/3/action/package_show?id=elev-30m-digital-elevation-model',directory + '\\MN_DEM',"GDB")

In [23]:
# Add DEM to map
m.addDataFromPath(directory + '\\MN_DEM\\elev_30m_digital_elevation_model.gdb\\digital_elevation_model_30m')

## Quality assurance of rasters

The code below develops a function which determines potentially incorrect or inaccurate information regarding the digital elevation model.

The two functions created below function:
- If the spatial resolution matches or is less than the preferred maximum spatial resolution
- If the DEM coordinate system does not match the map's coordinate system
    - If not, the raster is reprojected to match the map's coordinate system
- Analyze if there are any cells with no data
- Analyze if the bounding box of the raster is within or matches the bounding box of the state polygon
- Analyze if the lowest and highest cell values exceed the lowest and highest elevations of Minnesota respectively (if the raster is a DEM)

If the coordinate system of the raster does not match the map's coordinate system, the cell directly below performs a reprojection to match the coordinate systems. Otherwise, the functions simply print out information about the raster, labeling potential risks within the data. Any potential issues are labeled with "Warning:" at the beginning. 

### Function help

- in_raster - Raster dataset the operations will be performed on.
- out_raster - Raster dataset which will be output once the operations are finished.
- preferred_max_cellsize - The users preferred maximum spatial resolution in meters for the "in_raster".
- mn_fc - The polygon feature class of the state of Minnesota used within the project.
- raster_type (optional) - Type "DEM" for digital elevation model to check for correct elevations.
- elevation_units (optional) - Type "FEET" if the DEM uses feet for elevation units and "METERS" if the DEM uses meter for elevation units.

In [24]:
# Define a function which reprojects and checks for correct cellsize
def rast_sr_reproject(in_raster, out_raster, preferred_max_cellsize):
    
    # Get correct x and y cellsizes in meters - MN Geospatial Commons uses a UTM coordinate system, so cellsize is in meters
    cellsize_x = float(arcpy.management.GetRasterProperties(
        in_raster = in_raster,
        property_type = 'CELLSIZEX'
    ).getOutput(0))
    
    cellsize_y = float(arcpy.management.GetRasterProperties(
        in_raster = in_raster,
        property_type = 'CELLSIZEY'
    ).getOutput(0))
    
    # Print a warning if the maximum spatial resolution exceeds the preferred spatial resolution
    if cellsize_x >= cellsize_y:
        if preferred_max_cellsize > cellsize_x:
            print('Warning: Spatial resolution of raster exceeds your preferred spatial resolution. Raster may not be accurate enough for your analysis.')
        else:
            print('Spatial resolution of raster is less than or equal to your preferred spatial resolution.')
            
    if cellsize_y > cellsize_x:
        if preferred_max_cellsize > cellsize_y:
            print('Warning: Spatial resolution of raster exceeds your preferred spatial resolution. Raster may not be accurate enough for your analysis.')
        else:
            print('Spatial resolution of raster is less than or equal to your preferred spatial resolution.')
    
    # Set variables for the factory code of the spatial reference for both the map and raster
    map_epsg = spatial_ref.factoryCode
    rast_epsg = arcpy.Describe(in_raster).spatialReference.factoryCode
    
    # If coordinate systems do not match, reproject raster and delete the old one
    if map_epsg != rast_epsg:
        print('Coordinate system of raster does not match coordinate system of map. Reprojecting...')
        
        arcpy.management.ProjectRaster(
            in_raster = in_raster,
            out_raster = out_raster,
            out_coor_system = spatial_ref
        )
        
        arcpy.management.Delete(
            in_data = in_raster
        )
        
        print('Done')
        
    else:
        print('Coordinate system of raster matches coordinate system of map.')

In [25]:
# Run the function with the DEM
rast_sr_reproject('digital_elevation_model_30m','MN_DEM_30',30)

Spatial resolution of raster is less than or equal to your preferred spatial resolution.
Coordinate system of raster does not match coordinate system of map. Reprojecting...
Done


In [26]:
# Define another function for analyzing any issues with the data itself
def raster_check(in_raster, mn_fc, raster_type=None, elevation_units=None):
    
    # If the raster is a DEM, check if cells exceed the minimum and/or maximum elevations of Minnesota
    if raster_type == 'DEM':
    
        highest_elev = float(arcpy.management.GetRasterProperties(
            in_raster = in_raster,
            property_type = 'MAXIMUM'
        ).getOutput(0))

        lowest_elev = float(arcpy.management.GetRasterProperties(
            in_raster = in_raster,
            property_type = 'MINIMUM'
        ).getOutput(0))

        if elevation_units == 'FEET':

            if highest_elev > 2301:
                print('Warning: One or more DEM cell values exceed the maximum elevation of state.')
            if lowest_elev < 602:
                print('Warning: One or more DEM cell values are lower than the minimum elevation of state.')
            else:
                print("Elevation values are within state's lowest and highest point.")

        if elevation_units == 'METERS':

            if highest_elev > 701:
                print('Warning: One or more DEM cell values exceed the maximum elevation of state.')
            if lowest_elev < 183.5:
                print('Warning: One or more DEM cell values are lower than the minimum elevation of state.')
            else:
                print("Elevation values are within state's lowest and highest point.")
    
    else:
        print('Raster is not a DEM, no elevation check needed.')
    
    # Check if there are any cells with null values and print a warning if there are
    any_no_data = arcpy.management.GetRasterProperties(
        in_raster = in_raster,
        property_type = 'ANYNODATA'
    )
    
    if any_no_data == '1':
        print('Warning: DEM contains null values. It is recommended to perform a manual check on the raster.')
    else:
        print('No null values found.')
    
    # Create variables for the bounding box of the state feature class
    state_fc_desc = arcpy.Describe(mn_fc)
    fc_xmin = state_fc_desc.extent.XMin
    fc_xmax = state_fc_desc.extent.XMax
    fc_ymin = state_fc_desc.extent.YMin
    fc_ymax = state_fc_desc.extent.YMax
    
    # Create variables for the bounding box of the input raster
    rast_xmin = float(arcpy.management.GetRasterProperties(
        in_raster = in_raster,
        property_type = 'LEFT'
    ).getOutput(0))
    
    rast_xmax = float(arcpy.management.GetRasterProperties(
        in_raster = in_raster,
        property_type = 'RIGHT'
    ).getOutput(0))
    
    rast_ymin = float(arcpy.management.GetRasterProperties(
        in_raster = in_raster,
        property_type = 'BOTTOM'
    ).getOutput(0))
    
    rast_ymax = float(arcpy.management.GetRasterProperties(
        in_raster = in_raster,
        property_type = 'TOP'
    ).getOutput(0))
    
    # If the raster bounding box is larger than the state bounding box, print a warning
    if rast_xmin < fc_xmin or rast_ymin < fc_ymin or rast_xmax > fc_xmax or rast_ymax > fc_ymax:
        print('Warning: Raster bounding box exceeds state bounding box.')
    else:
        print('Raster is within state bounding box.')

In [27]:
# Run the function described above
raster_check('MN_DEM_30', 'Minnesota', 'DEM', 'FEET')

No null values found.


## Quality assurance check of DEM

- DEM has a spatial resolution which is less than or equal to the preferred spatial resolution of the user.
- The coordinate system was not WGS 84, so the DEM was reprojected.
- There are some cell values within the DEM which are less than the lowest elevation in the state of Minnesota. Cell values are multiples of 5 so this is OK.
- No null values in any cell in the DEM.
- The raster bounding box exceeds the bounding box of the state of Minnesota. This is OK.

In [28]:
# Pull and unzip land cover data from the MN Geospatial Commons
mn_geo_pull_and_unzip('https://gisdata.mn.gov/api/3/action/package_show?id=biota-landcover-nlcd-mn-2006',directory + "\\Land_Cover","GDB")

In [29]:
# Add land cover raster
m.addDataFromPath(directory + '\\Land_Cover\\biota_landcover_nlcd_mn_2006.gdb\\nlcd_mn_2006_land_cover')

In [30]:
# Run the reproject and spatial resolution check function
rast_sr_reproject('nlcd_mn_2006_land_cover', 'Land_Cover_reproject', 30)

Spatial resolution of raster is less than or equal to your preferred spatial resolution.
Coordinate system of raster does not match coordinate system of map. Reprojecting...
Done


In [31]:
# Run the raster data check function
raster_check('Land_Cover_reproject','Minnesota')

Raster is not a DEM, no elevation check needed.
No null values found.


## Quality assurance check of Land Cover raster

- Land Cover raster has a spatial resolution which is less than or equal to the preferred spatial resolution of the user.
- The coordinate system was not WGS 84, so the raster was reprojected.
- Land Cover raster does not use elevation data, so an elevation check is not needed.
- No null values in any cell in the DEM.
- The raster bounding box exceeds the bounding box of the state of Minnesota. This is ok.

In [32]:
# Saving a raster to an enterprise GDB can be diffcult, so the rasters will be converted to vector data

# Resample rasters for raster to point processing (9 kilometers is roughly 0.0278 vertical degrees)
arcpy.management.Resample(
    in_raster = 'MN_DEM_30',
    out_raster = 'res_MN_DEM',
    cell_size = 0.0278,
    resampling_type = 'BILINEAR'
)

arcpy.management.Resample(
    in_raster = 'Land_Cover_reproject',
    out_raster = 'res_Land_Cover',
    cell_size = 0.0278,
    resampling_type = 'BILINEAR'
)

# Convert raster to point for DEM data
arcpy.conversion.RasterToPoint(
    in_raster = 'res_MN_DEM',
    out_point_features = 'MN_DEM_pts',
    raster_field = 'Value'
)

# Convert raster to polygon for land cover data
arcpy.conversion.RasterToPolygon(
    in_raster = 'res_Land_Cover',
    out_polygon_features = 'Land_Cover_polygons',
    raster_field = 'Value'
)

<Result 'C:\\Users\\15612\\Documents\\ArcGIS\\Projects\\ArcII Lab 2\\ArcII Lab 2.gdb\\Land_Cover_polygons'>

### Once this notebook has been run, move to the "Weather Station and BMSB QA" notebook